<a href="https://colab.research.google.com/github/ihksans/lab-ai-group-4/blob/master/Group4_AI_Assignment_25042025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tugas Assigment Pertemuan 25-04-2025
- IKHSAN SETIAWAN (2802544545)
- AZHAR BANI RASHIF (2802544551)

In [2]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load dataset sentimen Indonesia (IndoNLU - SMSA)
dataset = load_dataset("indonlp/indonlu", "smsa")

# Load tokenizer IndoBERT
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/32.5k [00:00<?, ?B/s]

indonlu.py:   0%|          | 0.00/32.7k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/38.7k [00:00<?, ?B/s]

The repository for indonlp/indonlu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/indonlp/indonlu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

library yang dibutuhkan:

*   datasets dari HuggingFace untuk memuat dataset Indonesia
*   transformers untuk model dan tokenizer IndoBERT
*   sklearn untuk menghitung metrik evaluasi,
*   torch untuk representasi tensor saat inferensi nanti.

Kita menggunakan dataset SMSA (Sentiment Analysis) dari IndoNLU. Dataset ini berisi kalimat dalam Bahasa Indonesia yang sudah diberi label sentimen:


*   0: Negatif
*   1: Netral
*   2: Positif

Tokenizer ini berasal dari model IndoBERT pre-trained (indobenchmark/indobert-base-p1). Tokenizer ini akan mengubah teks menjadi token yang dapat dipahami model transformer.

In [5]:
# Fungsi tokenisasi
def tokenize_batch(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

# Tokenisasi seluruh dataset
tokenized = dataset.map(tokenize_batch, batched=True)

# Rename kolom dan ubah format
tokenized = tokenized.remove_columns(["text"])
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format("torch")

# Split dataset
train_dataset = tokenized["train"]
test_dataset = tokenized["validation"]


Map:   0%|          | 0/11000 [00:00<?, ? examples/s]

Di sini kita melakukan beberapa langkah penting:
*   Tokenisasi setiap kalimat, di-batch agar cepat.
*   Menghapus kolom text karena tidak lagi dibutuhkan setelah tokenisasi.
*   Mengganti nama kolom label ke labels, karena Trainer dari HuggingFace mengharuskan nama itu.
*   mengubah dataset ke format PyTorch.
Dataset dibagi menjadi dua:
*   train_dataset: data untuk training model.
*   test_dataset: data untuk evaluasi model. Kita ambil dari split validation, karena split test tidak menyediakan label.

In [6]:
# Load model IndoBERT untuk klasifikasi (3 label: negatif, netral, positif)
model = AutoModelForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1", num_labels=3)

# Fungsi evaluasi metrik
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Kita load model IndoBERT yang sudah pre-trained dan menambahkan head klasifikasi dengan 3 label.



Fungsi **compute_metrics** ini digunakan untuk mengevaluasi performa model. Kita hitung:

*   Akurasi
*   Precision
*   Recall
*   F1 Score

menggunakan strategi average macro karena jumlah data antar label bisa tidak seimbang.

In [8]:
# Konfigurasi training
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
)

# Inisialisasi Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset.select(range(2000)),  # Subset data kecil
    eval_dataset=test_dataset.select(range(500)),
    compute_metrics=compute_metrics,
)


Kita atur hyperparameter seperti:

*   Batch size
*   Jumlah epoch (di sini hanya 2, agar cepat)
*   Learning rate

Kemudian kita buat objek Trainer yang otomatis menangani training, evaluasi, dan logging.

In [9]:
# Training
trainer.train()

# Evaluasi hasil
eval_results = trainer.evaluate()
print(eval_results)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dreamweaver-main (dreamweaver-main-binus-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.203399,0.934000,0.910407,0.915465,0.912777
2,0.222800,0.208001,0.950000,0.934740,0.941273,0.937957


{'eval_loss': 0.20800091326236725, 'eval_accuracy': 0.95, 'eval_precision': 0.9347399072647079, 'eval_recall': 0.9412730127015841, 'eval_f1': 0.9379573831186735, 'eval_runtime': 191.2774, 'eval_samples_per_second': 2.614, 'eval_steps_per_second': 0.329, 'epoch': 2.0}


Kita jalankan proses training dan evaluasi model. Hasilnya akan menampilkan metrik seperti akurasi dan F1 score.

In [10]:
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

for sample in dataset["validation"].select(range(5)):
    inputs = tokenizer(sample["text"], return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    print(sample["text"][:100], "... Prediction:", label_map[pred])

meski masa kampanye sudah selesai , bukan berati habis pula upaya mengerek tingkat kedipilihan elekt ... Prediction: Neutral
tidak enak ... Prediction: Positive
restoran ini menawarkan makanan sunda . kami memesan ayam goreng , kangkung , sayur asam , ikan gura ... Prediction: Negative
lokasi di alun alun masakan padang ini cukup terkenal dengan kepala ikan kakap gule , biasa saya pes ... Prediction: Negative
betapa bejad kader gerindra yang anggota dprd mencabuli anak smp , rakyat harus cerdas partai mana y ... Prediction: Positive


Berikut adalah analisis common error cases dan language-specific challenges saat menggunakan model BERT (IndoBERT) untuk analisis sentimen dalam Bahasa Indonesia:

**Common Error Cases (Kesalahan Umum)**
1. Prediksi Netral vs Positif/Negatif
*   Banyak kasus di mana model memprediksi label "Netral" padahal secara konteks kalimat mengandung sentimen.
Contoh:
> "Barangnya bagus, tapi pengiriman lama."

> Kesalahan ini terjadi karena kalimat mengandung dua sentimen, dan model kesulitan mengambil dominan

2.  Penggunaan Bahasa Gaul / Singkatan
*   Model terkadang gagal memahami kata-kata gaul atau singkatan informal yang sering muncul di media sosial.
*   Contoh:

> "dptnya cepet bgt padahal COD, top dah seller"

> Kata seperti "dptnya", "bgt", "top dah" bisa menyulitkan jika tidak muncul cukup sering dalam pretraining corpus.

**Language-Specific Challenges (Tantangan Bahasa Spesifik Bahasa Indonesia**


1.   Morfologi yang Fleksibel
Bahasa Indonesia memiliki struktur morfologis yang fleksibel, misalnya prefiks/sufiks:

> bermain, dimainkan, pemain — semuanya dari akar kata main. ➤ Ini bisa menyebabkan sparseness atau noise jika model tidak cukup peka terhadap morfologi.

2. Kode-Mixing (Campuran Bahasa)
 mencampur Bahasa Indonesia dengan Bahasa Inggris:


> "Produknya oke sih, tapi shipping-nya slow banget."

3. Data Ketidakseimbangan Label

Seringkali jumlah label tidak seimbang (positif > netral > negatif), menyebabkan model bias terhadap mayoritas kelas.

 Kita tampilkan sebagian isi teks beserta label prediksi model (Positive, Neutral, atau Negative).

Link Google Collab: https://colab.research.google.com/drive/1FZUj5UOURrWKcB0DyOf2BiLsjuw_XE4t?usp=sharing